In [2]:
import pandas as pd
import numpy as np 

C:\Users\shonn\AppData\Local\Temp\ipykernel_16004\3482817184.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# 데이터 불러오기

df21 = pd.read_csv('result - 2021.csv')
df22 = pd.read_csv('result - 2022.csv')
df23 = pd.read_csv('result - 2023.csv')

In [4]:
# 팀명/팀점수 분리하기

def team_name(df) : 
    df['원정'] = df['원정'].str.replace("KIA", "기아", 1)
    df['홈'] = df['홈'].str.replace("KIA", "기아", 1)
    df['원정'] = df['원정'].str.replace("SSG", "SK", 1)
    df['홈'] = df['홈'].str.replace("SSG", "SK", 1)
    
def game_def(df) : 
    df['원정팀'] = df['원정'].str[:2]
    df['A'] = df['원정'].str[2:]
    df['홈팀'] = df['홈'].str[-2:]
    df['B'] = df['홈'].str[:-2]
    
    df['A'] = pd.to_numeric(df['A'])
    df['B'] = pd.to_numeric(df['B'])
    
    df['result'] = df['B'] - df['A']
    
        

In [5]:
team_name(df21)
team_name(df22)
team_name(df23)

game_def(df21)
game_def(df22)
game_def(df23)

In [6]:
# 게임년도 추가하기
 
df21['year'] = 2021
df22['year'] = 2022
df23['year'] = 2023

In [7]:
# 년/월/일/요일 분리하기

def date(df) :
    df['month'] = df['날짜'].str[:2]
    df['day'] = df['날짜'].str[3:5]
    df['week'] = df['날짜'].str[6:7]
    
    df['year'] = df['year'].astype(str)
    df['date'] = df['year'] + df['month'] + df['day']
    
        

date(df21)
date(df22)
date(df23)


In [8]:
# 경기결과 분류하기

def result(df) :
    df['승리팀'] = df['result'].fillna('우천취소')

    df.loc[df['result'] >0, '승리팀'] = df['홈팀']
    df.loc[df['result'] <0, '승리팀'] = df['원정팀']
    df.loc[df['result'] == 0, '승리팀'] = '무승부'
    
result(df21)
result(df22)
result(df23)

In [9]:
# 홈/원정 분류하기

def home_away(df) : 
    df.loc[df['승리팀'] == df['홈팀'], '승리팀구분'] = 'Home'
    df.loc[df['승리팀'] != df['홈팀'], '승리팀구분'] = 'Away'

home_away(df21)
home_away(df22)
home_away(df23)

In [10]:
# 경기코드 생성
def gamecode(df) : 
        df['code'] = df['date'] + df['원정팀'].str[:1] + df['홈팀'].str[:1]

gamecode(df21)
gamecode(df22)
gamecode(df23)

In [11]:
# 데이터프레임 합치기

kbo_df = pd.concat([df21, df22, df23], axis = 0)
kbo_df

,날짜,원정,홈,구장,비고,원정팀,A,홈팀,B,result,year,month,day,week,date,승리팀,승리팀구분,code
0,04.03(토),기아,두산,잠실,우천취소,기아,NaN,두산,NaN,NaN,2021,04,03,토,20210403,우천취소,Away,20210403기두
1,04.03(토),롯데,SK,문학,우천취소,롯데,NaN,SK,NaN,NaN,2021,04,03,토,20210403,우천취소,Away,20210403롯S
2,04.03(토),LG,NC,창원,우천취소,LG,NaN,NC,NaN,NaN,2021,04,03,토,20210403,우천취소,Away,20210403LN
3,04.03(토),한화,KT,수원,우천취소,한화,NaN,KT,NaN,NaN,2021,04,03,토,20210403,우천취소,Away,20210403한K
4,04.03(토),삼성1,6키움,고척,-,삼성,1.0,키움,6.0,5.0,2021,04,03,토,20210403,키움,Home,20210403삼키
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,10.16(월),SK3,2두산,잠실,-,SK,3.0,두산,2.0,-1.0,2023,10,16,월,20231016,SK,Away,20231016S두
809,10.16(월),NC2,4기아,광주,-,NC,2.0,기아,4.0,2.0,2023,10,16,월,20231016,기아,Home,20231016N기
810,10.16(월),롯데7,2한화,대전,-,롯데,7.0,한화,2.0,-5.0,2023,10,16,월,20231016,롯데,Away,20231016롯한
811,10.17(화),두산0,5SK,문학,-,두산,0.0,SK,5.0,5.0,2023,10,17,화,20231017,SK,Home,20231017두S


In [12]:
cond = ['code', 'date', 'week', '원정팀', '홈팀', 'A', 'B', '승리팀', '승리팀구분']
final_df = kbo_df[cond]
final_df

,code,date,week,원정팀,홈팀,A,B,승리팀,승리팀구분
0,20210403기두,20210403,토,기아,두산,NaN,NaN,우천취소,Away
1,20210403롯S,20210403,토,롯데,SK,NaN,NaN,우천취소,Away
2,20210403LN,20210403,토,LG,NC,NaN,NaN,우천취소,Away
3,20210403한K,20210403,토,한화,KT,NaN,NaN,우천취소,Away
4,20210403삼키,20210403,토,삼성,키움,1.0,6.0,키움,Home
...,...,...,...,...,...,...,...,...,...
808,20231016S두,20231016,월,SK,두산,3.0,2.0,SK,Away
809,20231016N기,20231016,월,NC,기아,2.0,4.0,기아,Home
810,20231016롯한,20231016,월,롯데,한화,7.0,2.0,롯데,Away
811,20231017두S,20231017,화,두산,SK,0.0,5.0,SK,Home


In [13]:
final_df.to_csv('kbo_winlose_final.csv')